# Анализ эффективности удержания

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%pylab inline

Populating the interactive namespace from numpy and matplotlib


### 1
В этом задании вам предлагается проанализировать данные одной из американских телекоммуникационных компаний о пользователях, которые потенциально могут уйти.

Измерены следующие признаки:

- state — штат США
- account_length — длительность использования аккаунта
- area_code — деление пользователей на псевдорегионы, использующееся в телекоме
- intl_plan — подключена ли у пользователя услуга международного общения
- vmail_plan — подключена ли у пользователя услуга голосовых сообщений
- vmail_message — количество голосых сообщений, который пользователь отправил / принял
- day_calls — сколько пользователь совершил дневных звонков
- day_mins — сколько пользователь проговорил минут в течение дня
- day_charge — сколько пользователь заплатил за свою дневную активность
- eve_calls, eve_mins, eve_charge — аналогичные метрики относительно вечерней активности
- night_calls, night_mins, night_charge — аналогичные метрики относительно ночной активности
- intl_calls, intl_mins, intl_charge — аналогичные метрики относительно международного общения
- custserv_calls — сколько раз пользователь позвонил в службу поддержки
- treatment — номер стратегии, которая применялись для удержания абонентов (0, 2 = два разных типа воздействия, 1 = контрольная группа)
- mes_estim — оценка интенсивности пользования интернет мессенджерами
- churn — результат оттока: перестал ли абонент пользоваться услугами оператора

Давайте рассмотрим всех пользователей из контрольной группы (treatment = 1). Для таких пользователей мы хотим проверить гипотезу о том, что штат абонента не влияет на то, перестанет ли абонент пользоваться услугами оператора.

Для этого мы воспользуемся критерием хи-квадрат. Постройте таблицы сопряженности между каждой из всех 1275 возможных неупорядоченных пар штатов и значением признака churn. Для каждой такой таблицы 2x2 применить критерий хи-квадрат можно с помощью функции scipy.stats.chi2_contingency(subtable, correction=False)

Заметьте, что, например, (AZ, HI) и (HI, AZ) — это одна и та же пара. Обязательно выставьте correction=False (о том, что это значит, вы узнаете из следующих вопросов).

Сколько достигаемых уровней значимости оказались меньше, чем α=0.05?

In [45]:
data = pd.read_csv('churn_analysis.csv', sep = ',')

In [46]:
data.head()

,Unnamed: 0,state,account_length,area_code,intl_plan,vmail_plan,vmail_message,day_mins,day_calls,day_charge,...,night_mins,night_calls,night_charge,intl_mins,intl_calls,intl_charge,custserv_calls,treatment,mes_estim,churn
0,0,KS,128,415,no,yes,25,265.1,110,45.07,...,244.7,91,11.01,10.0,3,2.70,1,1,0.65,False.
1,1,OH,107,415,no,yes,26,161.6,123,27.47,...,254.4,103,11.45,13.7,3,3.70,1,0,0.55,False.
2,2,NJ,137,415,no,no,0,243.4,114,41.38,...,162.6,104,7.32,12.2,5,3.29,0,0,0.72,False.
3,3,OH,84,408,yes,no,0,299.4,71,50.90,...,196.9,89,8.86,6.6,7,1.78,2,1,0.28,False.
4,4,OK,75,415,yes,no,0,166.7,113,28.34,...,186.9,121,8.41,10.1,3,2.73,3,2,0.45,False.


In [47]:
import scipy

In [48]:
states = list(set(data.state))
states

['MT',
 'VT',
 'IL',
 'AR',
 'OK',
 'KY',
 'NY',
 'ID',
 'DC',
 'NE',
 'WA',
 'IA',
 'NC',
 'TN',
 'TX',
 'RI',
 'MA',
 'WY',
 'AZ',
 'NV',
 'LA',
 'CA',
 'AK',
 'VA',
 'FL',
 'SD',
 'OH',
 'KS',
 'HI',
 'WV',
 'WI',
 'MN',
 'MI',
 'AL',
 'OR',
 'DE',
 'ME',
 'UT',
 'NH',
 'CT',
 'GA',
 'MD',
 'PA',
 'NM',
 'IN',
 'SC',
 'NJ',
 'ND',
 'MS',
 'CO',
 'MO']

In [49]:
for i in range(len(data)):
    if (data.churn[i] == 'False.'):
        data.churn[i] = 0
    else:
        data.churn[i] = 1
data.churn

/Users/alina/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/alina/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


0       0
1       0
2       0
3       0
4       0
       ..
3328    0
3329    0
3330    0
3331    0
3332    0
Name: churn, Length: 3333, dtype: object

In [50]:
sum(data.churn)

483

In [51]:
data_tr = data[data.treatment == 1]
data_tr

,Unnamed: 0,state,account_length,area_code,intl_plan,vmail_plan,vmail_message,day_mins,day_calls,day_charge,...,night_mins,night_calls,night_charge,intl_mins,intl_calls,intl_charge,custserv_calls,treatment,mes_estim,churn
0,0,KS,128,415,no,yes,25,265.1,110,45.07,...,244.7,91,11.01,10.0,3,2.70,1,1,0.65,0
3,3,OH,84,408,yes,no,0,299.4,71,50.90,...,196.9,89,8.86,6.6,7,1.78,2,1,0.28,0
8,8,LA,117,408,no,no,0,184.5,97,31.37,...,215.8,90,9.71,8.7,4,2.35,1,1,0.50,0
12,12,IA,168,408,no,no,0,128.8,96,21.90,...,141.1,128,6.35,11.2,2,3.02,1,1,0.37,0
17,17,VT,93,510,no,no,0,190.7,114,32.42,...,129.6,121,5.83,8.1,3,2.19,3,1,0.84,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3319,3319,WY,89,415,no,no,0,115.4,99,19.62,...,280.9,112,12.64,15.9,6,4.29,3,1,0.37,0
3322,3322,MD,62,408,no,no,0,321.1,105,54.59,...,180.5,72,8.12,11.5,2,3.11,4,1,0.63,1
3323,3323,IN,117,415,no,no,0,118.4,126,20.13,...,227.0,56,10.22,13.6,3,3.67,5,1,0.55,1
3325,3325,OH,78,408,no,no,0,193.4,99,32.88,...,243.3,109,10.95,9.3,4,2.51,2,1,0.65,0


In [55]:
from collections import Counter
Counter(data_tr.state)

Counter({'KS': 25,
         'OH': 24,
         'LA': 15,
         'IA': 15,
         'VT': 23,
         'CO': 22,
         'WY': 22,
         'MO': 21,
         'GA': 21,
         'AK': 20,
         'MA': 24,
         'NJ': 24,
         'AR': 16,
         'MI': 22,
         'IN': 23,
         'MN': 21,
         'SD': 23,
         'TX': 20,
         'DC': 19,
         'NY': 33,
         'OR': 21,
         'AZ': 19,
         'MS': 24,
         'UT': 22,
         'ME': 28,
         'ID': 18,
         'FL': 20,
         'NV': 24,
         'WA': 23,
         'WV': 32,
         'NH': 12,
         'VA': 23,
         'NM': 29,
         'KY': 25,
         'MT': 26,
         'PA': 18,
         'RI': 22,
         'DE': 17,
         'OK': 13,
         'NE': 24,
         'CT': 30,
         'WI': 24,
         'CA': 15,
         'NC': 19,
         'MD': 24,
         'TN': 12,
         'AL': 30,
         'IL': 12,
         'ND': 24,
         'HI': 13,
         'SC': 21})

In [56]:
len(states)

51

In [77]:
tables = []
for i, s1 in enumerate(states):
    for k, s2 in enumerate(states):
        if (k > i):
            table =[]
            t1 = data_tr[(data_tr.state == s1) & (data_tr.churn == 1)].shape[0]
            t2 = data_tr[(data_tr.state == s1) & (data_tr.churn == 0)].shape[0]
            t3 = data_tr[(data_tr.state == s2) & (data_tr.churn == 1)].shape[0]
            t4 = data_tr[(data_tr.state == s2) & (data_tr.churn == 0)].shape[0]
        
            table.insert(0, [t1, t2])
            table.insert(1, [t3, t4])
            tables.append(table)

In [81]:
from scipy import stats

In [83]:
psum = 0
for t in tables:
    if (scipy.stats.chi2_contingency(t, correction= False)[1] < 0.05):
        psum += 1
psum

34

In [84]:
psum = 0
for t in tables:
    if (scipy.stats.chi2_contingency(t, correction= True)[1] < 0.05):
        psum += 1
psum

0

In [85]:
psum = 0
for t in tables:
    if (scipy.stats.fisher_exact(t)[1] < 0.05):
        psum += 1
psum

10

In [87]:
data[['day_calls', 'mes_estim']].corr()

,day_calls,mes_estim
day_calls,1.000000,-0.051794
mes_estim,-0.051794,1.000000


In [89]:
data[['day_calls', 'mes_estim']].corr(method = 'spearman')

,day_calls,mes_estim
day_calls,1.00000,0.04335
mes_estim,0.04335,1.00000


In [96]:
table = np.array([])
t1 = data_tr[(data_tr.state == 'KS') & (data_tr.churn == 1)].shape[0] 
t2 = data_tr[(data_tr.state == 'KS') & (data_tr.churn == 0)].shape[0] 
table = (np.append(t1, t2)).reshape((2,1))

for s in states: 
    if (s != 'KS'):
        t1 = data_tr[(data_tr.state == s) & (data_tr.churn == 1)].shape[0] 
        t2 = data_tr[(data_tr.state == s) & (data_tr.churn == 0)].shape[0] 
        k = (np.append(t1, t2)).reshape((2,1)) 
        table = np.concatenate((table, k), axis=1) 

table

array([[ 7,  4,  5,  1,  5,  2,  4,  6,  2,  1,  3,  7,  2,  3,  1,  6,
         4,  7,  2,  2,  6,  0,  5,  1,  3,  2,  3,  2,  1,  5,  2,  2,
         5,  5,  3,  1,  8,  4,  2,  5,  2,  5,  4,  2,  3,  5,  4,  3,
         4,  5,  4],
       [18, 22, 18, 11, 11, 11, 21, 27, 16, 18, 21, 16, 13, 16, 11, 14,
        18, 17, 20, 17, 18, 15, 10, 19, 20, 18, 20, 22, 12, 27, 22, 19,
        17, 25, 18, 16, 20, 18, 10, 25, 19, 19, 14, 27, 20, 16, 20, 21,
        20, 17, 17]])

In [97]:
def cramers_stat(confusion_matrix): 
    chi2 = scipy.stats.chi2_contingency(confusion_matrix, correction= True)[0]
    n = confusion_matrix.sum() 
    return np.sqrt(1.*chi2 / (n*(min(confusion_matrix.shape)-1))) 

print("Cramer V: ", cramers_stat(table), " p-value: ", 
       scipy.stats.chi2_contingency(table, correction= True)[1])

Cramer V:  0.20039321502033322  p-value:  0.7097590042778467


In [98]:
def proportions_diff_confint_ind(sample1, sample2, alpha = 0.05):    
    z = scipy.stats.norm.ppf(1 - alpha / 2.)
    
    p1 = float(sum(sample1)) / len(sample1)
    p2 = float(sum(sample2)) / len(sample2)
    
    left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    
    return (left_boundary, right_boundary)

def proportions_diff_z_stat_ind(sample1, sample2):
    n1 = len(sample1)
    n2 = len(sample2)
    
    p1 = float(sum(sample1)) / n1
    p2 = float(sum(sample2)) / n2 
    P = float(p1*n1 + p2*n2) / (n1 + n2)
    
    return (p1 - p2) / np.sqrt(P * (1 - P) * (1. / n1 + 1. / n2))

def proportions_diff_z_test(z_stat, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided':
        return 2 * (1 - scipy.stats.norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less':
        return scipy.stats.norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - scipy.stats.norm.cdf(z_stat)

In [99]:
print("95%% confidence interval for a difference between proportions: [%f, %f]" %\
      proportions_diff_confint_ind(data[data.treatment == 0].churn, data[data.treatment == 1].churn))

95% confidence interval for a difference between proportions: [-0.048489, 0.011583]


In [100]:
print("95%% confidence interval for a difference between proportions: [%f, %f]" %\
      proportions_diff_confint_ind(data[data.treatment == 0].churn, data[data.treatment == 2].churn))

95% confidence interval for a difference between proportions: [-0.007821, 0.048856]


In [101]:
print("95%% confidence interval for a difference between proportions: [%f, %f]" %\
      proportions_diff_confint_ind(data[data.treatment == 1].churn, data[data.treatment == 2].churn))

95% confidence interval for a difference between proportions: [0.009619, 0.068322]
